In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time
import pymysql

api_key = '?api_key=****'

def getAllItems (prefix, store_name, suffix, flags = '' ):
    num_of_items = getItemCount(prefix, store_name, suffix, flags)
    print ('=== Fetching ' + str(num_of_items) + " items of " + suffix + " for " + store_name + ' ===')
    df_result = getPageItems(prefix, store_name, suffix, flags, page = 1) 
    for page in range (2, 1 + int(np.ceil(num_of_items / 100))):               
        results_n = getPageItems(prefix, store_name, suffix, flags, page) 
        df_result = df_result.append(results_n)
    print ('Done')
    return df_result

def getPageItems (prefix, store_name, suffix, flags , page):
    print("Getting page " + str(page))
    time.sleep(2)
    response = getattr(requests, 'get')('https://openapi.etsy.com' + prefix + store_name +
                                        suffix + api_key + flags +
                                       '&limit=100&offset=' + str((page - 1) * 100))
    r = response.json()
    return pd.DataFrame(r['results'])

def getItemCount (prefix, store_name, suffix, flags):
    response = getattr(requests, 'get')('https://openapi.etsy.com' + prefix + store_name +
                                        suffix + api_key + flags +
                                       '&limit=1' )
    r = response.json()
    return r['count']

def getUserFromStoreName( store_name):
    response = getattr(requests, 'get')('https://openapi.etsy.com' + '/v2/shops/' + store_name +
                                        api_key + 
                                       '&fields=user_id&limit=1' )
    r = response.json()
    return str(r['results'][0]['user_id'])

def getPerformanceForStores (stores_list_filename = 'stores_list.txt'):
    stores = pd.read_csv(stores_list_filename)
    df_perf = pd.DataFrame(columns=['user_id', 'store_name', 'timestamp', 'views', 'favorites', 'sales'])

    for store_name in stores.store_name:
        user_id = getUserFromStoreName(store_name)
        df_sales = getAllItems( '/v2/users/', user_id, '/profile')
        df_listings = getAllItems('/v2/shops/', store_name, '/listings/active' )
        output_filename = './' + store_name + '_' + time.strftime("%Y-%m-%d") +'.csv'
        df_sales['views'] = df_listings.views.sum().astype(str)
        df_sales['favorites'] = df_listings.num_favorers.sum().astype(str)
        df_sales['sales'] = df_sales['transaction_sold_count'].astype(str)
        df_sales['store_name'] = store_name
        df_sales['timestamp'] = (int(time.time() / 1000) * 1000)
        df = df_sales[['user_id', 'store_name', 'timestamp', 'views', 'favorites', 'sales']]
        df_perf['user_id'] = df_perf['user_id'].astype(float).astype(int)
        df_perf['timestamp'] = df_perf['timestamp'].astype(float).astype(int)
        df_perf['views'] = df_perf['views'].astype(int)  
        df_perf['favorites'] = df_perf['favorites'].astype(int)  
        df_perf['sales'] = df_perf['sales'].astype(int)  
        df_perf = df_perf.append(df)
    df_perf.reset_index(drop=True, inplace=True)
    return df_perf


def generateOutputQuery(df, db_table_name):
    
    for col_name in df_perf.columns:
        df_perf[col_name] = df_perf[col_name].astype(str)  
    df_perf['store_name'] = "'" + df_perf['store_name'] + "'"
    query = 'INSERT INTO ' + db_table_name + ' (user_id, login_name, `timestamp`, views, favorites, sales) VALUES ' 

    for row in range (0, len (df_perf)):
        query = query  + ' ( '
        query = query + ", ".join(list((df_perf.ix[row].values)))
        query = query + ' ) '
        if row == len(df_perf) - 1:
            query = query + ';'
        else: query = query + ','
    return query

def executeQuery(query):
    conn = pymysql.connect(host='HOST', port=3306, user='USER', passwd='PASSWD', db='DB')
    conn.autocommit(True)
    cur = conn.cursor()
    res = cur.execute(query)
    
    for row in cur:
        print(row)
    cur.close()
    conn.close()
    
def writeDFtoDatabase(df):
    print('Writing results to database')
    insert_query = generateOutputQuery(df, 'database1.stores_data')
    executeQuery(insert_query)
  

In [2]:
df_perf = getPerformanceForStores('all_stores.txt')
writeDFtoDatabase(df_perf)